In [2]:
import pandas as pd
import numpy as np
import os

### 1.数据基本清洗

#### 1.1读取json

In [6]:
os.getcwd()
os.chdir(r'D:\rui\code_analysis\homework\Project\art_crawl\article\article\spiders')
os.getcwd()

'D:\\rui\\code_analysis\\homework\\Project\\art_crawl\\article\\article\\spiders'

In [8]:
df = pd.read_json('resault.json')
len(df)

1652

#### 1.1.1少了keywords列，爬取后合并

In [84]:
df2 = pd.read_json('keywords.json')
df2[df2['title'].str.contains('新型城镇化的关键改革与突')]

,title,date,keywords,fund
3,新型城镇化的关键改革与突破口选择,2015年01期,"[新型城镇化, 户籍制度, 城中村土地制度改革, 工业用地重整, 农地整理]",[]


In [169]:
df3 = pd.merge(df,df2,on='title')

In [98]:
df3.drop(['keywords_x','fund_x','date_x'],axis=1,inplace=True)
df3.rename(columns={'date_y':'date','keywords_y':'keywords','fund_y':'fund'},inplace=True)

#### 1.2清除无摘要/关键词/作者等非论文行

In [ ]:
#删除空白行
def tr(x):
    return len(x) != 0
df3 = df3[df.keywords.apply(tr)]
df4 = df3.reset_index(drop=True).copy()
len(df4)

#### 1.3部分列数据清洗

In [117]:
df4.date[0]

'2015年01期'

In [120]:
#date只截取年份
int(df4.date[0].split('年')[0])

2015

In [121]:
df4.date = df4.date.apply(lambda x:int(x.split('年')[0]))
df4.date[0]

2015

In [ ]:
#author_num只截取数量
int(df4.author_num[0].split('：')[1])

In [81]:
df4.author_num = df4.author_num.apply(lambda x:int(x.split('：')[1]))

AttributeError: 'int' object has no attribute 'split'

In [65]:
#fund列处理，替换空白行为np.nan

In [ ]:
df4.fund = df4.fund.apply(lambda x:np.nan if len(x) == 0 else x )

In [130]:
df4.to_excel('resault.xlsx',sheet_name='first')

In [131]:
df4.to_json('resault_second.json')

#### 1.4增加需要的列

#### 1.4.1作者对文章贡献度

In [146]:
round(1.0/len(df4[df4['title'] == df4['title'][854]]),2)

0.14

In [147]:
df4['participation'] = df4['title'].apply(lambda x:round(1.0/len(df4[df4['title'] == x]),2))

#### 1.4.2作者所在省市

In [154]:
#对机构进行去重
ins_check = df4[['ins']]

In [155]:
ins_check = ins_check.drop_duplicates().reset_index(drop=True)

In [159]:
ins_check['province'] = np.nan
ins_check['city'] = ins_check.ins.apply(lambda x:x[:2])
ins_check['ins_mo'] = ins_check.ins
ins_check

,ins,province,city,ins_mo
0,中国人民大学,NaN,中国,中国人民大学
1,东南大学,NaN,东南,东南大学
2,上海市城市规划设计研究院,NaN,上海,上海市城市规划设计研究院
3,山东建筑大学,NaN,山东,山东建筑大学
4,伦敦学院大学,NaN,伦敦,伦敦学院大学
...,...,...,...,...
237,湖南大学设计研究院有限公司,NaN,湖南,湖南大学设计研究院有限公司
238,中国电子信息产业发展研究院,NaN,中国,中国电子信息产业发展研究院
239,广东财经大学,NaN,广东,广东财经大学
240,启迪设计集团股份有限公司,NaN,启迪,启迪设计集团股份有限公司


In [160]:
ins_check.to_excel('ins_check.xlsx')

In [161]:
ins_mo = pd.read_excel('ins_check.xlsx')

In [162]:
author_use =  pd.merge(df4,ins_mo,on='ins')

In [166]:
author_use.drop(['ins_mo'],axis=1,inplace=True)

In [168]:
author_use.to_json('author_use.json')